In [1]:
from requests_html import HTMLSession
from concurrent.futures import ThreadPoolExecutor
import requests
import os
import shutil
import glob
import pandas as pd
import dask.dataframe as dd
import dask.multiprocessing
import dask.threaded
from dask.diagnostics import ProgressBar
ProgressBar().register()

In [2]:
def download_file(url):
    response = requests.get(url)
    if "content-disposition" in response.headers:
        content_disposition = response.headers["content-disposition"]
        filename = content_disposition.split("filename=")[1]
    else:
        filename = url.split("/")[-1]
        with open(filename, mode="wb") as file:
            file.write(response.content)
        print(f"Downloaded file {filename}")

In [3]:
# Arquivos a partir de 2021 - ainda passam pelo processo de atualização CSV
session = HTMLSession()
links = []
base_url = 'https://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/'
r = session.get(base_url)
print(r)

for link in r.html.links:
    if not link.endswith('.zip'):
        continue
    url = base_url+link
    links.append(url)

<Response [200]>


In [4]:
for link in links:
    print('Baixando', link)
    download_file(link)

Baixando https://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_202108.zip
Downloaded file inf_diario_fi_202108.zip
Baixando https://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_202208.zip
Downloaded file inf_diario_fi_202208.zip
Baixando https://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_202107.zip
Downloaded file inf_diario_fi_202107.zip
Baixando https://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_202201.zip
Downloaded file inf_diario_fi_202201.zip
Baixando https://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_202210.zip
Downloaded file inf_diario_fi_202210.zip
Baixando https://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_202306.zip
Downloaded file inf_diario_fi_202306.zip
Baixando https://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_202309.zip
Downloaded file inf_diario_fi_202309.zip
Baixando https://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_202209.zip
D

In [5]:
print('Movendo arquivos zip para a pasta ./zip')
for file_name in os.listdir():
    if not file_name.endswith('.zip'):
        continue
    if not os.path.exists(os.path.join('.', 'zip')):
        os.mkdir('zip')
    print(file_name)
    shutil.unpack_archive(file_name)
    shutil.move(
        os.path.join(file_name),
        os.path.join('zip', file_name)
    )
print('ok')

Movendo arquivos zip para a pasta ./zip
inf_diario_fi_202207.zip
inf_diario_fi_202105.zip
inf_diario_fi_202306.zip
inf_diario_fi_202108.zip
inf_diario_fi_202303.zip
inf_diario_fi_202210.zip
inf_diario_fi_202212.zip
inf_diario_fi_202107.zip
inf_diario_fi_202205.zip
inf_diario_fi_202208.zip
inf_diario_fi_202301.zip
inf_diario_fi_202302.zip
inf_diario_fi_202103.zip
inf_diario_fi_202308.zip
inf_diario_fi_202307.zip
inf_diario_fi_202104.zip
inf_diario_fi_202102.zip
inf_diario_fi_202310.zip
inf_diario_fi_202106.zip
inf_diario_fi_202109.zip
inf_diario_fi_202304.zip
inf_diario_fi_202203.zip
inf_diario_fi_202305.zip
inf_diario_fi_202101.zip
inf_diario_fi_202309.zip
inf_diario_fi_202202.zip
inf_diario_fi_202401.zip
inf_diario_fi_202211.zip
inf_diario_fi_202110.zip
inf_diario_fi_202311.zip
inf_diario_fi_202201.zip
inf_diario_fi_202112.zip
inf_diario_fi_202111.zip
inf_diario_fi_202206.zip
inf_diario_fi_202204.zip
inf_diario_fi_202209.zip
inf_diario_fi_202312.zip
ok


In [6]:
print('Movendo arquivos csv para a pasta ./csv')
for file_name in os.listdir():
    if not file_name.endswith('.csv'):
        continue
    if not os.path.exists(os.path.join('.', 'csv')):
        os.mkdir('csv')
    print(file_name)
    shutil.move(
        os.path.join(file_name),
        os.path.join('csv', file_name)
    )
print('ok')

Movendo arquivos csv para a pasta ./csv
inf_diario_fi_202105.csv
inf_diario_fi_202211.csv
inf_diario_fi_202109.csv
inf_diario_fi_202101.csv
inf_diario_fi_202401.csv
inf_diario_fi_202304.csv
inf_diario_fi_202303.csv
inf_diario_fi_202209.csv
inf_diario_fi_202104.csv
inf_diario_fi_202305.csv
inf_diario_fi_202307.csv
inf_diario_fi_202206.csv
inf_diario_fi_202301.csv
inf_diario_fi_202201.csv
inf_diario_fi_202208.csv
inf_diario_fi_202207.csv
inf_diario_fi_202205.csv
inf_diario_fi_202111.csv
inf_diario_fi_202306.csv
inf_diario_fi_202204.csv
inf_diario_fi_202210.csv
inf_diario_fi_202309.csv
inf_diario_fi_202308.csv
inf_diario_fi_202310.csv
inf_diario_fi_202110.csv
inf_diario_fi_202103.csv
inf_diario_fi_202311.csv
inf_diario_fi_202106.csv
inf_diario_fi_202112.csv
inf_diario_fi_202312.csv
inf_diario_fi_202302.csv
inf_diario_fi_202202.csv
inf_diario_fi_202212.csv
inf_diario_fi_202107.csv
inf_diario_fi_202102.csv
inf_diario_fi_202203.csv
inf_diario_fi_202108.csv
ok


In [7]:
from timeit import default_timer as timer
start = timer()

# Faz o tratamento das colunas
df = dd.read_csv('csv/inf_diario_fi_*.csv', sep=";")

# transforma o campo DT_COMPTC
df['CNPJ_FUNDO'] = df['CNPJ_FUNDO'].str.replace('.', '')
df['CNPJ_FUNDO'] = df['CNPJ_FUNDO'].str.replace('/', '')
df['CNPJ_FUNDO'] = df['CNPJ_FUNDO'].str.replace('-', '')
df['CNPJ_FUNDO'] = df['CNPJ_FUNDO'].str.zfill(14)

# cria o campo CO_PRD
df['CO_PRD'] = df['CNPJ_FUNDO']

# cria e formata o campo DT_REF, com a data de referência
print('Formatando campo DT_REF')
df.assign(DT_COMPTC=dd.to_datetime(df['DT_COMPTC'], format='%Y-%m-%d', errors='coerce'))
df['DT_REF'] = df['DT_COMPTC']

Formatando campo DT_REF


In [8]:
# faz o sort do df
df.sort_values(by=['CO_PRD', 'DT_REF'], inplace=True)
df.tail()

[########################################] | 100% Completed | 157.27 s
[########################################] | 100% Completed | 924.51 ms


,TP_FUNDO,CNPJ_FUNDO,DT_COMPTC,VL_TOTAL,VL_QUOTA,VL_PATRIM_LIQ,CAPTC_DIA,RESG_DIA,NR_COTST,CO_PRD,DT_REF
411582,FI,97929213000134,2024-01-17,86999223.09,11.602655,83540214.19,0.0,0.0,2,97929213000134,2024-01-17
411583,FI,97929213000134,2024-01-18,86999223.09,11.602655,83540214.19,0.0,0.0,2,97929213000134,2024-01-18
411584,FI,97929213000134,2024-01-19,86999223.09,11.602655,83540214.19,0.0,0.0,2,97929213000134,2024-01-19
411585,FI,97929213000134,2024-01-22,86536084.41,11.537926,82933867.36,0.0,0.0,2,97929213000134,2024-01-22
411586,FI,97929213000134,2024-01-23,86741878.92,11.566530,83280108.87,0.0,0.0,2,97929213000134,2024-01-23


In [9]:
# Convert to a Pandas DataFrame because dask was being slow with the select logic below
print('Converte dataframe dask para pandas')
df = df.compute() 

# cria uma nova coluna com o percentual de resgate para o dia
print('Calculando campo PC_RESG')
df['PC_RESG'] = df['RESG_DIA'] / df.groupby(['CO_PRD'])['VL_PATRIM_LIQ'].shift(1)

# Convert back to a Dask dataframe because we want that juicy parallelism
print('Converte dataframe pandas para dask')
df = dd.from_pandas(df, npartitions=4)

df.tail()

Converte dataframe dask para pandas
[########################################] | 100% Completed | 62.57 s
Calculando campo PC_RESG
Converte dataframe pandas para dask
[########################################] | 100% Completed | 102.63 ms


,TP_FUNDO,CNPJ_FUNDO,DT_COMPTC,VL_TOTAL,VL_QUOTA,VL_PATRIM_LIQ,CAPTC_DIA,RESG_DIA,NR_COTST,CO_PRD,DT_REF,PC_RESG
578707,FI,97929213000134,2023-08-25,84412931.79,11.107729,84411205.20,0.0,0.0,2,97929213000134,2023-08-25,0.0
578708,FI,97929213000134,2023-08-28,84459070.59,11.113726,84456776.59,0.0,0.0,2,97929213000134,2023-08-28,0.0
578709,FI,97929213000134,2023-08-29,84695430.03,11.144754,84692568.40,0.0,0.0,2,97929213000134,2023-08-29,0.0
578710,FI,97929213000134,2023-08-30,84718705.72,11.147742,84715275.32,0.0,0.0,2,97929213000134,2023-08-30,0.0
578711,FI,97929213000134,2023-08-31,84559196.05,11.126677,84555196.78,0.0,0.0,2,97929213000134,2023-08-31,0.0


In [10]:
# salva num único csv
df.to_csv('2021_em_diante.dask', sep=";", index=False)
print('csv de saída salvo com sucesso')

end = timer()
print(end - start)

[########################################] | 100% Completed | 297.72 s
csv de saída salvo com sucesso
532.2129690520014
